References:<br>
https://docs.azuredatabricks.net/user-guide/secrets/secrets.html<br>
https://docs.azuredatabricks.net/spark/latest/data-sources/azure/azure-storage.html<br>
https://docs.azuredatabricks.net/user-guide/dbfs-databricks-file-system.html<br>

Notes:
DBFS commands are very specific. Some hard-coding/specific formats -required-.

Prefer Azure Key Vault for secret storage.<br>
Create Azure Key Vault, store secret there. Then create ADB secret scope.<br>
Then use secret scope and storage acct info to mount storage acct/container to DBFS mount point.

In [2]:
storageAcctName = "pxbrixsa"
containerName = "hack"

secretScopeName = "pzbrixscope"
secretName = "pzbrixsakey"

mountPoint = "/mnt/" + containerName

In [3]:
# Explicit version works but hard-codes container and storage account names - have not been able to get this to work with variables and string concat
dbutils.fs.mount(
  source = "wasbs://hack@pzbrixsa.blob.core.windows.net",
  mount_point = mountPoint,
  extra_configs = {"fs.azure.account.key.pzbrixsa.blob.core.windows.net":dbutils.secrets.get(scope = secretScopeName, key = secretName)}
)

In [4]:
# This (or a version of the above using variables) does not work. Gets java.lang.IllegalArgumentException due to invalid mount source. ??
dbutils.fs.mount(
        "wasbs://{cn}@{san}.blob.core.windows.net"
        .format(
            cn=containerName,
            san=storageAcctName),
        "/mnt/{mn}".format(mn=mountPoint)
    )

In [5]:
# ls in the newly mounted mount point / in the mounted container
display(dbutils.fs.ls("/mnt/" + containerName))

In [6]:
# Unmount the storage mount point
dbutils.fs.unmount("/mnt/" + containerName)

In [7]:
# Refresh mounts on other clusters than the one that ran DBFS commands
dbutils.fs.refreshMounts()

In [8]:
display(dbutils.fs.ls("/mnt"))